# Langchain

Langchain is a framework designed for working with LLM's to create powerfull applications powerd by AI. It works as an interface between the llm models and other tools and applciations. It allows us to create complex interactions between llm's, API's and different data sources

## LLM Wrappers

Load API keys

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

Read env keys

In [2]:
openai_api_key = os.getenv("OPEN_API_KEY")
hugging_face_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
langchain_token = os.getenv("LANGCHAIN_API_KEY")

pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_env = os.getenv("PINECONE_ENV")
pinecone_index_host = os.getenv("PINECONE_INDEX_HOST")


### LLM Wrappers

In [3]:
from langchain_huggingface import HuggingFaceEndpoint

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Initialize the HuggingFaceEndpoint
llm = HuggingFaceEndpoint(repo_id=repo_id,
                          max_length=128,
                          temperature=0.3,
                          huggingfacehub_api_token=hugging_face_token
                          )

# Use the llm instance as needed
response = llm.invoke("Who won the FIFA World Cup in the year 1994?")
print(response)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\Hori\.cache\huggingface\token
Login successful


The FIFA World Cup in the year 1994 was won by the Brazilian national football team. Brazil defeated Italy in the final match, which was held on July 17, 1994, at the Rose Bowl in Pasadena, California, USA. Brazil's star player, Romário, was the top scorer of the tournament, with five goals. Brazil won their fourth World Cup title with this victory.


#### Chat models

In [4]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)
from langchain_huggingface import ChatHuggingFace

messages = [
    SystemMessage(content="You are a sports expert"),
    HumanMessage(content="Who won the world cup in 1994?")
]

chat_model = ChatHuggingFace(llm=llm)


In [5]:
chat_model.model_id

'mistralai/Mistral-7B-Instruct-v0.2'

In [6]:
chat_model._to_chat_prompt(messages)

'<s> [INST] You are a sports expert\n\nWho won the world cup in 1994? [/INST]'

In [7]:
res = chat_model.invoke(messages)
print(res.content)

 I'm here to help answer your sports-related questions. The 1994 FIFA World Cup was won by the Brazilian national soccer team. They defeated Italy in the final match held in the United States, with a score of 0-0 after regular time, 3-2 after penalty kicks. This title marked Brazil's fourth World Cup win, a record they share with Italy. The team, led by star players like Romário and Dunga, put


#### Prompt Templates

In [8]:
from langchain import PromptTemplate

template = """
    You are an expert european footbal commentator.
    Explain who won the  world cup in {world_cup_year}. and why they were successful.
"""

prompt = PromptTemplate(
    input_variables = ["world_cup_year"],
    template = template,
)

prompt

PromptTemplate(input_variables=['world_cup_year'], template='\n    You are an expert european footbal commentator.\n    Explain who won the  world cup in {world_cup_year}. and why they were successful.\n')

In [9]:
llm(prompt.format(world_cup_year="2002"))

c:\Users\Hori\Desktop\Evozon\python\llm\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


"\nI'd be happy to help you with that!\n\nThe 2002 FIFA World Cup was won by the Brazilian national football team. This was their fifth World Cup title, making them the most successful team in the history of the tournament.\n\nBrazil's success in 2002 can be attributed to several factors. First and foremost was their exceptional attacking play, which was spearheaded by superstars Ronaldo and Rivaldo. Ronaldo, who had missed the 1998 World Cup due to health issues, was in top form and scored eight goals in the tournament, including two in the final against Germany. Rivaldo also chipped in with five goals.\n\nBrazil's midfield was another strength, with the likes of Kaká, Ronaldinho, and Edmílson providing the creativity and energy that drove the team forward. The defense, led by captain Cafu and center back Lucio, was solid and rarely gave away cheap goals.\n\nBrazil's tactical flexibility was also a key factor in their success. Manager Luiz Felipe Scolari employed a 3-5-2 formation in 

#### Chains

Takes a language model and a prompt template and combines them to an interface that takes an input from the user and outputs an answer from the llm - sort of like a composite function where the inner function is the prompt template and the outer function is the language model

We can also build composite chains where one chain outputs and answer and than we have a second chain that takes the output of the first chain as an input

In [10]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

#Run the chain only specifying the input variable
print(chain.run("2002"))


I'd be happy to help you with that!

The 2002 FIFA World Cup was won by the Brazilian national football team. This was their fifth World Cup title, making them the most successful team in the history of the tournament.

Brazil's success in 2002 can be attributed to several factors. First and foremost was their exceptional attacking play, which was spearheaded by superstars Ronaldo and Rivaldo. Ronaldo, who had missed the 1998 World Cup due to health issues, was in top form and scored eight goals in the tournament, including two in the final against Germany. Rivaldo also chipped in with five goals.

Brazil's midfield was another strength, with the likes of Kaká, Ronaldinho, and Edmílson providing the creativity and energy that drove the team forward. The defense, led by captain Cafu and center back Lucio, was solid and rarely gave away cheap goals.

Brazil's tactical flexibility was also a key factor in their success. Manager Luiz Felipe Scolari employed a 3-5-2 formation in the knocko

c:\Users\Hori\Desktop\Evozon\python\llm\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\Hori\Desktop\Evozon\python\llm\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [11]:
second_prompt = PromptTemplate(
    input_variables = ["year"],
    template = "Who was the best player of that world cup in {year} of that team that won the world cup",
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [12]:
third_prompt = PromptTemplate(
    input_variables = ["year"],
    template = "What was the average posession of the team who won the world cup in {year}",
)

chain_three = LLMChain(llm=llm, prompt=third_prompt)

In [13]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[chain, chain_two, chain_three], verbose=True)

# Run the chain specifying only the input variable of the first chain
explanation = overall_chain.run("2010")
print(explanation)



> Entering new SimpleSequentialChain chain...

I'd be happy to help you with that!

The 2010 FIFA World Cup was won by Spain. This was a historic achievement for the Spanish national team, as they became the first European team to win the World Cup on South American soil.

Spain's success in the tournament can be attributed to several factors. First and foremost was their exceptional team play and passing ability. Spain's midfield, in particular, was a force to be reckoned with, with players like Xavi Hernandez, Andres Iniesta, and Sergio Busquets controlling the tempo of the game and creating scoring opportunities.

Another key factor was Spain's defensive solidity. They conceded just three goals in seven games, thanks in large part to the excellent performances of goalkeeper Iker Casillas and central defenders Carlos Puyol and Gerard Pique.

Spain also had a strong attacking threat, with players like David Villa and Fernando Torres capable of scoring goals. Villa, in particular, ha

In [14]:
print(explanation)



However, to answer your question, the average possession for Spain during the 2010 World Cup was 68.3%. This was the highest possession percentage in the tournament and was a testament to Spain's dominance of the ball and their ability to keep possession for long periods of time. This allowed them to control the game and wear down their opponents, making it difficult for them to mount effective attacks.


In [15]:
# Custom function to run each chain and collect their outputs
def run_chains_and_collect_outputs(overall_chain, input_data):
    outputs = []
    current_input = input_data
    for chain in overall_chain.chains:
        current_output = chain.run(current_input)
        outputs.append(current_output)
        current_input = current_output
    return outputs

# Run the custom function and collect outputs
all_outputs = run_chains_and_collect_outputs(overall_chain, "2010")

# The last output will be in the last position of the list
final_output = all_outputs[-1]

print(all_outputs)  # To see all outputs
print(final_output)  # To see the final output

["\nI'd be happy to help you with that!\n\nThe 2010 FIFA World Cup was won by Spain. This was a historic achievement for the Spanish national team, as they became the first European team to win the World Cup on South American soil.\n\nSpain's success in the tournament can be attributed to several factors. First and foremost was their exceptional team play and passing ability. Spain's midfield, in particular, was a force to be reckoned with, with players like Xavi Hernandez, Andres Iniesta, and Sergio Busquets controlling the tempo of the game and creating scoring opportunities.\n\nAnother key factor was Spain's defensive solidity. They conceded just three goals in seven games, thanks in large part to the excellent performances of goalkeeper Iker Casillas and central defenders Carlos Puyol and Gerard Pique.\n\nSpain also had a strong attacking threat, with players like David Villa and Fernando Torres capable of scoring goals. Villa, in particular, had a standout tournament, scoring five

Merge the responses

In [16]:
result = ', '.join(all_outputs)

#### Embeddings and vector stores

Split the text into chunks

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def get_recursive_text_splitter(chunk_size, chunk_overlap):
    return RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap,
    )
    
def split_documents(docs, text_splitter):
    return  text_splitter.create_documents([docs])


text_splitter = get_recursive_text_splitter(chunk_size=100, chunk_overlap=0)
splitted_docs = split_documents(result, text_splitter)

In [18]:
for text in splitted_docs:
    print(text.page_content)

I'd be happy to help you with that!
The 2010 FIFA World Cup was won by Spain. This was a historic achievement for the Spanish national
team, as they became the first European team to win the World Cup on South American soil.
Spain's success in the tournament can be attributed to several factors. First and foremost was
their exceptional team play and passing ability. Spain's midfield, in particular, was a force to be
reckoned with, with players like Xavi Hernandez, Andres Iniesta, and Sergio Busquets controlling
the tempo of the game and creating scoring opportunities.
Another key factor was Spain's defensive solidity. They conceded just three goals in seven games,
thanks in large part to the excellent performances of goalkeeper Iker Casillas and central
defenders Carlos Puyol and Gerard Pique.
Spain also had a strong attacking threat, with players like David Villa and Fernando Torres capable
of scoring goals. Villa, in particular, had a standout tournament, scoring five goals and being

### Create a datasest with the texts 

I will create a pandas dataset where I will insert the text and asign a unique id for each separate text

In [19]:
import pandas as pd
import uuid

# Function to generate unique IDs
def generate_unique_id():
    return str(uuid.uuid4())


# Extract the page_content from each Document object into a separate list
page_contents = [doc.page_content for doc in splitted_docs]

df = pd.DataFrame(page_contents, columns=["chunk"])

# Generate a unique ID for each row and add it as a new column in the DataFrame
df['unique_id'] = df.apply(lambda row: generate_unique_id(), axis=1)


Turn the text into Embeddings, first i need to import an embeddings model

In [20]:
from langchain_huggingface import HuggingFaceEmbeddings

small_embeddings_model = 'sentence-transformers/all-MiniLM-L6-v2'
normal_embeddings_model = 'sentence-transformers/all-mpnet-base-v2'

model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=normal_embeddings_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

Transform the chuntks in the dataset into embeddings

In [21]:
df['embedding'] = df['chunk'].apply(lambda text: embeddings.embed_query(text))

In [22]:
df.head()

,chunk,unique_id,embedding
0,I'd be happy to help you with that!,c882dbfe-5c69-41e5-87a3-adab202a41d1,"[0.0031824701000005007, 0.028275931254029274, ..."
1,The 2010 FIFA World Cup was won by Spain. This...,51d6ed3f-16f3-423b-af6e-c5a719e11647,"[0.011273376643657684, 0.024946296587586403, -..."
2,"team, as they became the first European team t...",d72b1952-a41a-47d7-a6b5-c48785a9da9b,"[-0.048796359449625015, -0.040353938937187195,..."
3,Spain's success in the tournament can be attri...,459a2818-776a-4c6f-ac2d-8834f751452a,"[0.025057660415768623, -0.004411689005792141, ..."
4,their exceptional team play and passing abilit...,9a6c47ae-841a-48a2-a18e-baf7dfd223e5,"[-0.006107549648731947, -0.03988241031765938, ..."


Connect to pinecone and create an index if it does not exist

In [24]:
import pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)

index_name = "test-index"
index_list = pc.list_indexes()
embedding_dimensions = len(df['embedding'][0])


def check_index_exists(data, index_name):
    indexes = data.get('indexes', [])
    for index in indexes:
        if index.get('name') == index_name:
            return True
    return False

# Check if the index already exists
if check_index_exists(index_list, index_name) is False:
    pc.create_index(
        name=index_name,
        dimension=embedding_dimensions, # Replace with your model dimensions
        metric="euclidean", # Replace with your model metric
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )

Grab the index from pinecone

In [25]:
index = pinecone.Index(api_key=pinecone_api_key,
                       host=pinecone_index_host,
                       index_name=index_name
                       )

Create a list of dictionaries for the data you want to upsert in the Pinecone vector store

In [26]:
data_to_upsert = [
    {
        "id": row['unique_id'],
        "values": row['embedding']
    }
    for index, row in df.iterrows()
]

Upload the mebeddings to the vector store

In [27]:
index.upsert(vectors=data_to_upsert)
print(f"{len(data_to_upsert)} documents successfully upserted to Pinecone index.")

29 documents successfully upserted to Pinecone index.


### Query the documents we have in the vector store

Query the pinecone vector store to find out the most simmilar docs given a question

Pause the exectuion to wait for pinecone

In [36]:
import time

#Stop for a few seconds so it can index the data in pinecone
time.sleep(10)

In [37]:
# Top k answers
top_k = 1

# Question
question = "What was the average posession?"
# Questin embeddings vector
query_vector = embeddings.embed_query(question)

results = index.query(
    vector=query_vector,
    top_k=top_k,
    include_values=True
)

# Iterate over matches and print document IDs and scores
print("Top {} similar documents:".format(len(results.matches)))
for idx, match in enumerate(results.matches):
    doc_id = match['id']
    score = match['score']
    print(f"{idx + 1}. Document ID: {doc_id}, Similarity Score: {score}")


Top 1 similar documents:
1. Document ID: 25d11145-0c0f-48c0-90ac-93dc0218cc26, Similarity Score: 1.31664336


### Match the resutls with the chunks I have in the dataframe

Now I need to associate the answers retunrned by the query with the original texts(chunks) from the dataframe

In [38]:
# Define the method to find text by UUID
def find_text_by_uuid(df, uuid, text_column='chunk', id_column='unique_id'):
    result = df[df[id_column] == uuid]
    if not result.empty:
        return result.iloc[0][text_column]
    else:
        return None

answers = []

for match in results.matches:
    answer = {
        'uuid': match['id'],
        'chunk': find_text_by_uuid(df, match['id']),
        'score': match['score']
    }
    answers.append(answer)

# Sort the answers    
answers = sorted(answers, key=lambda x: x['score'], reverse=True)

Sort the answers descendingly

In [39]:
texts_responses = [answer['chunk'] for answer in answers]
print(texts_responses)

["68.3%. This was the highest possession percentage in the tournament and was a testament to Spain's"]


Grab the first answer, the answer with the hightest score:

In [40]:
most_probable_response = texts_responses[0]
print(most_probable_response)

68.3%. This was the highest possession percentage in the tournament and was a testament to Spain's
